In [3]:
import numpy as np
import pandas as pd
# import scipy.stats as stats
import sklearn
import csv
import hashlib
import matplotlib.pyplot as plt

%matplotlib inline

def convertMovieName(ttid):
	return ttid[2:]

def convertPGRating(rating):
	#g, pg, pg-13, r, nc-17
	# rating = rating.lower()
	ratingDict = {
	    'P': 1,
	    'PG': 2,
	    'PG-13': 3,
	    'R': 4,
	    'NC-17': 5,
	}
	if rating not in ratingDict:
		return 0
	return ratingDict[str(rating).strip()]


converterFunctions = {}
converterFunctions[12] = convertPGRating
converterFunctions[1] = convertMovieName

dataset = pd.read_csv("data/FinalDataSet.csv", delimiter=",",usecols = [1,2,3,4,5,6,7,8,9,10,11,12], converters=converterFunctions)
print dataset.columns
print len(dataset)


Index([u'imdbID', u'Actor1_Rev', u'Actor2_Rev', u'Year', u'YT_Count',
       u'YT_Likes', u'YT_Dislike', u'IMDBVotes', u'imdbRating', u'Budget',
       u'Revenue', u'Rated'],
      dtype='object')
3488


In [4]:
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split(dataset, test_size = 0.2)

In [5]:
def getRevenueClass(budget,revenue):
    profit = int((revenue - budget)/float(budget))
    if profit == 0:
        return 0
    if profit == 1:
        return 1
    if profit == 2:
        return 2
    if profit == 3:
        return 3
    if profit >= 4:
        return 4
    return -1
        
    

In [6]:
import statsmodels.formula.api as smf
lm = smf.ols(formula='Revenue ~ Budget+YT_Count + Rated + IMDBVotes*IMDBVotes', data=trainData).fit()

preds = lm.predict(testData)
predictions = []

for value in preds:
    predictions.append(value)

counter = 0
correctCount = 0
revenueList = testData['Revenue']
budgetList = testData['Budget'].tolist()
for realValue in revenueList:
    predictedValue = predictions[counter]
#     diff = abs(realValue - predictedValue)
# #     print diff
#     percentError = diff/float(realValue)
# #     print str(percentError) + ":" + str(diff)
#     if percentError < 0.25:
#         correctCount = correctCount + 1
    budget = budgetList[counter]
    actualClass = getRevenueClass(budget,realValue)
    predictedClass = getRevenueClass(budget,predictedValue)
    if(actualClass == predictedClass):
        correctCount = correctCount + 1
    counter = counter + 1

print correctCount
print correctCount*100/float(698)

ValueError: cannot convert float NaN to integer

In [7]:
budgetList = testData['Budget'].tolist()
budgetList

[75000000,
 28000000,
 47000000,
 48000000,
 90000000,
 20000000,
 75000000,
 35000000,
 52000000,
 5000000,
 17000000,
 150000000,
 1800000,
 33000000,
 12000000,
 16000000,
 60000000,
 3000000,
 9000000,
 74000000,
 17500000,
 2000000,
 4500000,
 18000000,
 3600000,
 1000000,
 139000000,
 19500000,
 2883848,
 25000000,
 60000000,
 35000000,
 30000000,
 15000000,
 8500000,
 41000000,
 250000,
 35000000,
 6000000,
 25000000,
 155000000,
 14000000,
 100000000,
 34000000,
 10000000,
 5000000,
 23000000,
 2450000,
 40000000,
 6000000,
 37000000,
 5000000,
 600000,
 26000000,
 17700000,
 14000000,
 23000000,
 8000000,
 160000000,
 1000000,
 80000000,
 17000000,
 100000000,
 30000000,
 40000000,
 18000000,
 2000000,
 8000000,
 150000000,
 52000000,
 20000000,
 92000000,
 1000000,
 16500000,
 9000000,
 144000000,
 50000000,
 90000000,
 500000,
 25000000,
 14000000,
 40000000,
 29000000,
 26000000,
 3000000,
 2500000,
 15000000,
 22000000,
 38000000,
 35000000,
 30000000,
 200000000,
 5000000

In [8]:
lm.pvalues

Intercept     1.476740e-23
Budget        6.734762e-05
YT_Count      8.215389e-54
Rated         3.617218e-15
IMDBVotes    8.241407e-248
dtype: float64

In [29]:
lm.rsquared

0.49521199486580791

In [11]:
lm2 = smf.ols(formula='Revenue ~ Budget+YT_Count + Rated + IMDBVotes', data=trainData).fit()
lm2.pvalues

Intercept     1.476740e-23
Budget        6.734762e-05
YT_Count      8.215389e-54
Rated         3.617218e-15
IMDBVotes    8.241407e-248
dtype: float64

In [31]:
lm2.params


Intercept    6.581088e+07
Budget       3.599720e-02
YT_Count     3.232494e+00
Rated       -1.647305e+07
IMDBVotes    5.932714e+02
dtype: float64

In [32]:
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     682.8
Date:                Sun, 30 Apr 2017   Prob (F-statistic):               0.00
Time:                        18:26:20   Log-Likelihood:                -55689.
No. Observations:                2789   AIC:                         1.114e+05
Df Residuals:                    2784   BIC:                         1.114e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.581e+07   6.23e+06     10.559      0.000    5.36e+07     7.8e+07
Budget         0.0360      0.009      4.078      0.000       0.019       0.053
YT_Count       3.2325      0.186     17.386      0.000       2.868       3.597
Rated      -1.647e+07   1.89e+06     -8.734      0.000   -2.02e+07   -1.28e+07
IMDBVotes    593.2714     14.168     41.874      0.000     565.491     621.052
==============================================================================
Omnibus:                     1431.938   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31940.782
Skew:                           1.937   Prob(JB):                         0.00
Kurtosis:                      19.120   Cond. No.                     7.47e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.47e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""